# Apply a trained land classifier model in ArcGIS Pro

This tutorial will assume that you have already provisioned a [Geo AI Data Science Virtual Machine]() and are using this Jupyter notebook while connected via remote desktop on that VM. If not, please see our guide to [provisioning and connecting to a Geo AI DSVM]().

By default, this tutorial will make use of a model we have pre-trained for 250 epochs. If you have completed the associated notebook on [training a land classifier from scratch](./Train_a_land_classification_model_from_scratch.ipynb), you will have the option of using your own model file.

## Setup instructions

### Install any updates to ArcGIS Pro

At the time of this writing, updating ArcGIS Pro will install a new Python environment for use by the program. To simplify our instructions below, we will assume that you have updated ArcGIS Pro by following these instructions:

1. Search for and launch the ArcGIS Pro program.
1. When prompted, enter your username and password.
    - If you don't have an ArcGIS Pro license, see the instructions for getting a trial license in the [intro notebook](Intro_to_pixel-level_land_classification.ipynb).)
1. As the program loads, a software update notification may appear at upper-right.
    - If so, click on the notification for more information, then accept and start the software update. Once the update is complete, follow the prompted suggestion to restart ArcGIS Pro.

### Install the custom raster function

We will use Python scripts to apply a trained model to aerial imagery in real-time as the user scrolls through a region of interest in ArcGIS Pro. These Python scripts are surfaced in ArcGIS Pro as a [custom raster function](https://github.com/Esri/raster-functions). The three files needed for the raster function (the main Python script, helper functions for e.g. colorizing the model's results, and an XML description file) will be copied into the appropriate ArcGIS Pro subdirectory with the command below:

In [ ]:
!AzCopy /Source:https://aiforearthcollateral.blob.core.windows.net/imagesegmentationtutorial/arcgispro/ClassifyCNTK /SourceSAS:"?st=2018-01-16T10%3A40%3A00Z&se=2028-01-17T10%3A40%3A00Z&sp=rl&sv=2017-04-17&sr=c&sig=KeEzmTaFvVo2ptu2GZQqv5mJ8saaPpeNRNPoasRS0RE%3D" /Dest:C:\Program Files\ArcGIS\Pro\Resources\Raster\Functions\Custom\ClassifyCNTK /S

### Install the supporting files

If you have not already completed the associated notebook on [training a land classifier from scratch](./Train_a_land_classification_model_from_scratch.ipynb), execute the following cell to download supporting files to your Geo AI DSVM's D: drive.

In [ ]:
!AzCopy /Source:https://aiforearthcollateral.blob.core.windows.net/imagesegmentationtutorial /SourceSAS:"?st=2018-01-16T10%3A40%3A00Z&se=2028-01-17T10%3A40%3A00Z&sp=rl&sv=2017-04-17&sr=c&sig=KeEzmTaFvVo2ptu2GZQqv5mJ8saaPpeNRNPoasRS0RE%3D" /Dest:D:\pixellevellandclassification /S

You will now have a local copy of a sample ArcGIS Pro project, sample trained model file, and Python wheels for CNTK and its dependencies.

### Add ArcGIS Pro's Python environment to the path

At the time of this writing, ArcGIS Pro's software updates install a second Python environment containing Python 3.6. (The Geo AI DSVM's default Python environment, `py35`, then cannot be used.) We will need to add ArcGIS's python environment to the system path manually, as follows:

1. Click on the Start menu and type in `Edit the system environment variables`. Click on the eponymous search result to load the "Advanced" tab of the System Properties window.
1. In the window that appears, click the "Environment Variables..." button at lower-right.
1. In the "System variables" window, select the "PATH" entry and click the "Edit" button.
1. Click the "New" button. Type `C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3` into the new line, then click "OK."
1. Click "OK" to exit the Environment Variables window.
1. Click "OK" to exit the System Properties window.

### Install necessary Python packages in ArcGIS Pro's Python environment

Execute the following cell to install CNTK and its dependencies in the ArcGIS Pro Python 3.6 environment:

In [ ]:
!"C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\python.exe" -m pip install D:\pixellevellandclassification\wheels\numpy-1.14.0+mkl-cp36-cp36m-win_amd64.whl
!"C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\python.exe" -m pip install D:\pixellevellandclassification\wheels\scipy-1.0.0-cp36-cp36m-win_amd64.whl
!"C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\python.exe" -m pip install D:\pixellevellandclassification\wheels\cntk-2.0rc2-cp36-cp36m-win_amd64.whl

## Evaluate the model in real-time using ArcGIS Pro

### Load the sample project in ArcGIS Pro

Begin by loading the sample ArcGIS Pro project we have provided:

1. Search for and launch the ArcGIS Pro program 
1. On the ArcGIS Pro start screen, click on "Open an Existing Project".
1. Navigate to the folder where you extracted the sample project, and select the `D:\pixellevellandclassification\arcgispro\sampleproject.aprx` file. Click "OK."

Once the project has loaded, you should see a screen split into four quadrants. After a moment, NAIP aerial imagery and ground-truth land use labels should beome visible in the upper-left and upper-right corners, respectively. The bottom quadrants will show predictions from your trained model after you complete the following steps:

1. Begin by selecting the "AI Mixed Probabilities" window at bottom-left.
1. Add and modify an aerial imagery layer:
    1. In the Catalog Pane (accessible from the View menu), click on Portal, then the cloud icon (labeled "All Portal" on hover).
    1. In the search field, type NAIP.
    1. Drag and drop the "USA NAIP Imagery: Natural Color" option into the window at bottom-left. You should see a new layer with this name appear in the Contents Pane at left.
    1. Right-click on "USA NAIP Imagery: Natural Color" in the Contents Pane and select "Properties".
    1. In the "Processing Templates" tab of the layer properties, change the Processing Template from "Natural Color" to "None," then click OK.
1. Add a model predictions layer:
    1. In the Raster Functions Pane (accessible from the Analysis menu), you should see an entry named Classify under the "[ClassifyCNTK]" heading. (You may need to collapse and re-expand the heading.) Click this button to bring up the raster function options.
    1. Set the input raster to "USA NAIP Imagery: Natural Color".
    1. Set the trained model location to `D:\pixellevellandclassification\models\250epochs.model`.
        - Note: if you trained your own model using our companion notebook, you can use it instead by choosing `D:\pixellevellandclassification\models\trained.model` as the location.
    1. Set the output type to "Softmax", indicating that each pixel's color will be an average of the class label colors, weighted by their relative probabilities.
        - Note: selecting "Hardmax" will assign each pixel its most likely label's color insead.
    1. Click "Create new layer". After a few seconds, the model's predictions should appear in the bottom-left quadrant.
1. Repeat these steps with the bottom-right quadrant, selecting "Hardmax" as the output type.

Now that your project is complete, you can navigate and scroll to compare ground truth vs. predicted labels throughout the Chesapeake Bay watershed region. If you venture outside that area, you may find that ground truth regions are no longer available, but NAIP data and model predictions should still be displayed. 

## Next steps

In this notebook series, we trained and deployed a model on a Geo AI Data Science VM. To improve model accuracy, we recommend training for more epochs on a larger dataset. Please see [our GitHub repository](https://github.com/Azure/pixel_level_land_classification) for more details on scaling up training using Batch AI.

When you are done using your Geo AI Data Science VM, we recommend that you stop or delete it to prevent further charges.